In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-realized-volatility-prediction/sample_submission.csv
/kaggle/input/optiver-realized-volatility-prediction/train.csv
/kaggle/input/optiver-realized-volatility-prediction/test.csv
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=97/888f813404d8417ca8d6b8aebd5f2951.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=43/bb0efa57f511470e817880842e3e2afa.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=21/1d8dc18ebfee47ffbb54b04e6afc0634.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=72/60f62a03d8854605901dda072c84db39.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=4/761268d671f9429abb29d9d2895e9bd2.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=112/cd283097a5b54293ba400a19e811a7f9.parquet
/kaggle/input/optiver-realized-volatility-pr

1. IMPORTS & SEEDING

In [8]:
import os
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def seed_everything(seed=42):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

2. Looking At The Data

In [10]:
train_df = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
print(f"Train data loaded: {train_df.shape}")
print(f"   - Unique stocks: {train_df['stock_id'].nunique()}")
print(f"   - Unique time_ids: {train_df['time_id'].nunique()}")
print(f"   - Target range: [{train_df['target'].min():.6f}, {train_df['target'].max():.6f}]")

# sequential split by time_id
print("\n Creating sequential train/validation split...")
unique_time_ids = sorted(train_df['time_id'].unique())
split_point = int(0.8 * len(unique_time_ids))

train_time_ids = unique_time_ids[:split_point]
valid_time_ids = unique_time_ids[split_point:]

train_data = train_df[train_df['time_id'].isin(train_time_ids)].copy()
valid_data = train_df[train_df['time_id'].isin(valid_time_ids)].copy()

print(f"Sequential split completed:")
print(f"   - Train: {len(train_data)} rows ({len(train_time_ids)} time_ids)")
print(f"   - Valid: {len(valid_data)} rows ({len(valid_time_ids)} time_ids)")
print(f"   - Train time_id range: {min(train_time_ids)} to {max(train_time_ids)}")
print(f"   - Valid time_id range: {min(valid_time_ids)} to {max(valid_time_ids)}")


#valid short for validation


Train data loaded: (428932, 3)
   - Unique stocks: 112
   - Unique time_ids: 3830
   - Target range: [0.000105, 0.070321]

 Creating sequential train/validation split...
Sequential split completed:
   - Train: 343144 rows (3064 time_ids)
   - Valid: 85788 rows (766 time_ids)
   - Train time_id range: 5 to 25680
   - Valid time_id range: 25683 to 32767


In [11]:
book_train_path = '/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet'
trade_train_path = '/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet'

sample_book = pd.read_parquet(f'{book_train_path}/stock_id=0')
sample_trade = pd.read_parquet(f'{trade_train_path}/stock_id=0')

print(f"Sample book data (stock_id=0): {sample_book.shape}")
print(f"   Columns: {list(sample_book.columns)}")
print(f"Sample trade data (stock_id=0): {sample_trade.shape}")
print(f"   Columns: {list(sample_trade.columns)}")

Sample book data (stock_id=0): (917553, 10)
   Columns: ['time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']
Sample trade data (stock_id=0): (123443, 5)
   Columns: ['time_id', 'seconds_in_bucket', 'price', 'size', 'order_count']


3. Feature Engineering & Helper Functions

In [12]:
def wap(row):
    denom = row['ask_size1'] + row['bid_size1']
    return ((row['bid_price1'] * row['ask_size1'] + row['ask_price1'] * row['bid_size1']) / denom)


In [13]:
def wap2(row):
    denom = row['ask_size2'] + row['bid_size2'] 
    return ((row['bid_price2'] * row['ask_size2'] + row['ask_price2'] * row['bid_size2']) / denom)


In [14]:
def calculate_vwap(price, size):
    if size.sum() == 0:
        return 0
    return (price * size).sum() / size.sum()

In [15]:
def log_return(series):
    return np.log(series).diff()


In [16]:
def realized_volatility(series_log_return):
    if len(series_log_return) < 1:
        return 0
    return np.sqrt(np.sum(series_log_return**2))

In [17]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


In [18]:
def simple_volatility(series_prix):
    if len(series_prix) < 2:
        return 0
    mx = np.max(series_prix)
    mn = np.min(series_prix)
    moy = np.mean(series_prix)
    if (mx - mn) == 0:
        return 0
    vol = (moy - mn) / (mx - mn)
    return vol



In [19]:
def count_unique(series):
    return len(np.unique(series))

In [20]:
def book_features_advanced(book_df):
    if len(book_df) == 0:
        return {}
    #create copy
    df = book_df.copy()
    
    df['wap'] = df.apply(wap, axis=1)
    df['wap1'] = df.apply(wap2, axis=1)
    df['squared_wap'] = df['wap'] ** 2
    
    # Supply/demand features
    df['supply_demand'] = (df['ask_size1'] + df['ask_size2']) / (df['bid_size1'] + df['bid_size2'])
    
    # Average WAP
    df['avg_wap'] = (df['wap'] + df['wap1']) / 2
    
    # Price differences
    df['diff_bid_price'] = df['bid_price1'] - df['bid_price2']
    df['diff_ask_price'] = df['ask_price2'] - df['ask_price1']
    
    # WAP3 
    df['wap3'] = ((df['bid_price1'] * df['bid_size1']) + (df['ask_price1'] * df['ask_size1'])) / (df['bid_size1'] + df['ask_size1'])
    
    # WAP balance
    df['wap_balance'] = abs(df['wap'] - df['wap1']) + 1
    
    # Price spreads
    df['price_spread'] = df['ask_price1'] - df['bid_price1']
    df['price_spread_1'] = df['ask_price2'] - df['bid_price2']
    df['price_spread_avg'] = (df['price_spread'] + df['price_spread_1']) / 2
    df['price_spread_diff'] = df['price_spread_1'] - df['price_spread']
    
    # Volume features
    df['total_volume'] = df['ask_size1'] + df['bid_size1'] + df['ask_size2'] + df['bid_size2']
    df['volume_imbalance'] = abs((df['bid_size1'] + df['bid_size2']) - (df['ask_size1'] + df['ask_size2'])) + 1
    df['bid_size'] = df['bid_size1'] + df['bid_size2']
    df['ask_size'] = df['ask_size1'] + df['ask_size2']
    
    # Calculate log returns for time series 
    if len(df) > 1:
        df['log_return'] = log_return(df['wap'])
        df['log_return1'] = log_return(df['wap1'])
        df['log_squared_wap'] = log_return(df['squared_wap'])
        df['log_supply_demand'] = log_return(df['supply_demand'])
        df['log_avg_wap'] = log_return(df['avg_wap'])
        df['log_diff_ask_price'] = log_return(df['diff_ask_price'])
        df['log_diff_bid_price'] = log_return(df['diff_bid_price'])
        df['log_wap3'] = log_return(df['wap3'])
        df['log_wap_balance'] = log_return(df['wap_balance'])
        df['log_price_spread'] = log_return(df['price_spread'])
        df['log_price_spread_1'] = log_return(df['price_spread_1'])
        df['log_price_spread_avg'] = log_return(df['price_spread_avg'])
        df['log_price_spread_diff'] = log_return(df['price_spread_diff'])
        df['log_volume_imbalance'] = log_return(df['volume_imbalance'])
    else:
        # Set log returns to 0 if only one row
        log_cols = ['log_return', 'log_return1', 'log_squared_wap', 'log_supply_demand', 
                   'log_avg_wap', 'log_diff_ask_price', 'log_diff_bid_price', 'log_wap3',
                   'log_wap_balance', 'log_price_spread', 'log_price_spread_1', 
                   'log_price_spread_avg', 'log_price_spread_diff', 'log_volume_imbalance']
        for col in log_cols:
            df[col] = 0
    
    # Feature aggregation dictionary 
    create_feature_dict = {
        'log_return': [realized_volatility],
        'log_return1': [realized_volatility],
        'log_squared_wap': [realized_volatility],
        'log_supply_demand': [realized_volatility],
        'log_avg_wap': [realized_volatility],
        'supply_demand': [np.mean],
        'diff_bid_price': [np.mean],
        'diff_ask_price': [np.mean],
        'log_diff_ask_price': [realized_volatility],
        'log_diff_bid_price': [realized_volatility],
        'log_wap3': [realized_volatility],
        'wap_balance': [np.mean],
        'price_spread_1': [np.mean],
        'price_spread': [np.mean],
        'price_spread_avg': [np.mean],
        'price_spread_diff': [np.mean],
        'volume_imbalance': [np.mean],
        'total_volume': [np.mean],
        'wap': [np.mean],
        'log_wap_balance': [realized_volatility],
        'log_price_spread': [realized_volatility],
        'log_price_spread_1': [realized_volatility],
        'log_price_spread_avg': [realized_volatility],
        'log_price_spread_diff': [realized_volatility],
        'log_volume_imbalance': [realized_volatility],
        'bid_size': [np.sum],
        'ask_size': [np.sum]
    }
    
    # Calculate features according to the dictionary
    features = {}
    for feature_name, funcs in create_feature_dict.items():
        if feature_name in df.columns:
            for func in funcs:
                if func == realized_volatility:
                    # Remove NaN values for volatility calculation
                    clean_series = df[feature_name].dropna()
                    features[f'{feature_name}_realized_vol'] = func(clean_series) if len(clean_series) > 0 else 0
                else:
                    features[f'{feature_name}_{func.__name__}'] = func(df[feature_name])
    
    # Add count
    features['book_count'] = len(df)
    
    return features

In [21]:
def trade_features(trade_df):
    if len(trade_df) == 0:
        return {}
    
    # Calculate VWAP
    vwap = calculate_vwap(trade_df['price'], trade_df['size'])
    
    features = {
        # Trade price features
        'trade_price_mean': trade_df['price'].mean(),
        'trade_price_std': trade_df['price'].std(),
        'trade_price_min': trade_df['price'].min(),
        'trade_price_max': trade_df['price'].max(),
        
        # Trade size features
        'trade_size_mean': trade_df['size'].mean(),
        'trade_size_std': trade_df['size'].std(),
        'trade_size_min': trade_df['size'].min(),
        'trade_size_max': trade_df['size'].max(),
        'trade_size_sum': trade_df['size'].sum(),
        
        # Order count features
        'order_count_mean': trade_df['order_count'].mean(),
        'order_count_std': trade_df['order_count'].std(),
        'order_count_min': trade_df['order_count'].min(),
        'order_count_max': trade_df['order_count'].max(),
        'order_count_sum': trade_df['order_count'].sum(),
        
        # VWAP and volatility
        'trade_vwap': vwap,
        'trade_log_return_realized_vol': realized_volatility(trade_df['price']),
        
        # Count and activity measures
        'trade_count': len(trade_df),
        'trades_per_second': len(trade_df) / 600 if len(trade_df) > 0 else 0,  # 600 seconds = 10 minutes
    }
    
    return features

Aggregating per time window

In [22]:


def engineer_features_for_stock_time(stock_id, time_id):
  
    try:
        # Load book and trade data for this stock
        book_df = pd.read_parquet(f'{book_train_path}/stock_id={stock_id}')
        trade_df = pd.read_parquet(f'{trade_train_path}/stock_id={stock_id}')
        
        # Filter for specific time_id
        book_slice = book_df[book_df['time_id'] == time_id].copy()
        trade_slice = trade_df[trade_df['time_id'] == time_id].copy()
        
        # Engineer features using advanced methods
        book_feat = book_features_advanced(book_slice)
        trade_feat = trade_features(trade_slice)
        
        # Combine features
        all_features = {**book_feat, **trade_feat}
        all_features['stock_id'] = stock_id
        all_features['time_id'] = time_id
        
        return all_features
        
    except Exception as e:
        print(f"Error processing stock_id={stock_id}, time_id={time_id}: {e}")
        return None

# Test the function with one example
print("Testing feature engineering function...")

# Get a sample time_id for testing
sample_time_id = train_df['time_id'].iloc[0]  # Use first time_id from loaded data
print(f"   Using sample stock_id=0, time_id={sample_time_id}")

test_features = engineer_features_for_stock_time(0, sample_time_id)
if test_features:
    print(f"Test successful! Generated {len(test_features)} features:")
    # Show first few features
    feature_items = list(test_features.items())
    for key, value in feature_items[:8]:
        if isinstance(value, (int, float)):
            print(f"   - {key}: {value:.6f}")
        else:
            print(f"   - {key}: {value}")
    print(f"   - ... (and {len(test_features)-8} more features)")
else:
    print("Test failed!")


Testing feature engineering function...
   Using sample stock_id=0, time_id=5
Test successful! Generated 48 features:
   - log_return_realized_vol: 0.004499
   - log_return1_realized_vol: 0.006999
   - log_squared_wap_realized_vol: 0.008999
   - log_supply_demand_realized_vol: 23.882098
   - log_avg_wap_realized_vol: 0.004115
   - supply_demand_mean: 10.392625
   - diff_bid_price_mean: 0.00017551712517160922
   - diff_ask_price_mean: 0.00015085145423654467
   - ... (and 40 more features)


Correlation Analysis

In [23]:
def analyze_features_sample(sample_size=1000):

    print(f"Analyzing features using {sample_size} random samples...")

    # Get a random sample of (stock_id, time_id) combinations
    sample_data = train_df.sample(n=min(sample_size, len(train_df)), random_state=42)

    feature_rows = []
    targets = []

    print("Processing sample combinations...")
    for idx, (_, row) in enumerate(sample_data.iterrows()):
        if (idx + 1) % 200 == 0:
            print(f"   Processed {idx + 1}/{len(sample_data)} combinations...")

        # Ensure stock_id and time_id are integers
        stock_id = int(row['stock_id'])
        time_id = int(row['time_id'])
        target = row['target']

        # Engineer features for this combination
        features = engineer_features_for_stock_time(stock_id, time_id)

        if features:
            features_without_ids = {k: v for k, v in features.items()
                                  if k not in ['stock_id', 'time_id']}
            feature_rows.append(features_without_ids)
            targets.append(target)

    # Convert to DataFrame for analysis
    features_df = pd.DataFrame(feature_rows)
    targets_series = pd.Series(targets, name='target')

    print(f"Feature analysis complete! Processed {len(feature_rows)} valid combinations")
    print(f"   Generated {len(features_df.columns)} features")

    return features_df, targets_series


In [24]:
def correlation_analysis(features_df, targets_series):
    """Analyze correlations between features and target"""
    print("\n Computing feature-target correlations...")
    
    # Calculate correlations
    correlations = []
    for col in features_df.columns:
        # Handle potential NaN values
        valid_mask = ~(features_df[col].isna() | targets_series.isna())
        if valid_mask.sum() > 10:  # Need at least 10 valid points
            corr = np.corrcoef(features_df[col][valid_mask], targets_series[valid_mask])[0, 1]
            if not np.isnan(corr):
                correlations.append({'feature': col, 'correlation': abs(corr), 'correlation_raw': corr})
    
    # Sort by absolute correlation
    correlations_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)
    
    print(f"Correlation analysis complete!")
    print(f"\n TOP 15 MOST CORRELATED FEATURES:")
    print("="*60)
    for idx, row in correlations_df.head(15).iterrows():
        print(f"{row['feature']:<35} | {row['correlation_raw']:>8.4f} | {row['correlation']:>8.4f}")
    
    print(f"\n BOTTOM 10 LEAST CORRELATED FEATURES:")
    print("="*60)
    for idx, row in correlations_df.tail(10).iterrows():
        print(f"{row['feature']:<35} | {row['correlation_raw']:>8.4f} | {row['correlation']:>8.4f}")
    
    return correlations_df


In [25]:
def feature_statistics(features_df):
    """Analyze feature statistics"""
    print(f"\n Feature Statistics Summary:")
    print("="*60)
    
    # Check for features with too many NaN values
    nan_counts = features_df.isna().sum()
    high_nan_features = nan_counts[nan_counts > len(features_df) * 0.1]  # More than 10% NaN
    
    if len(high_nan_features) > 0:
        print(f"  Features with >10% missing values:")
        for feature, count in high_nan_features.items():
            pct = (count / len(features_df)) * 100
            print(f"   - {feature}: {count}/{len(features_df)} ({pct:.1f}%)")
    
    # Check for zero variance features
    zero_var_features = []
    for col in features_df.columns:
        if features_df[col].std() == 0 or features_df[col].std() < 1e-10:
            zero_var_features.append(col)
    
    if zero_var_features:
        print(f"\n  Zero/near-zero variance features (consider removing):")
        for feature in zero_var_features:
            print(f"   - {feature}")
    
    print(f"\n Total features analyzed: {len(features_df.columns)}")
    print(f" Features with high missing values: {len(high_nan_features)}")
    print(f" Zero variance features: {len(zero_var_features)}")


In [26]:
print("Starting feature analysis...")
sample_features_df, sample_targets = analyze_features_sample(sample_size=800)

# Analyze correlations
correlations_df = correlation_analysis(sample_features_df, sample_targets)

# Feature statistics
feature_statistics(sample_features_df)


Starting feature analysis...
Analyzing features using 800 random samples...
Processing sample combinations...
   Processed 200/800 combinations...
   Processed 400/800 combinations...
   Processed 600/800 combinations...
   Processed 800/800 combinations...
Feature analysis complete! Processed 800 valid combinations
   Generated 46 features

 Computing feature-target correlations...
Correlation analysis complete!

 TOP 15 MOST CORRELATED FEATURES:
log_return_realized_vol             |   0.8905 |   0.8905
log_squared_wap_realized_vol        |   0.8905 |   0.8905
log_wap3_realized_vol               |   0.8895 |   0.8895
log_return1_realized_vol            |   0.8868 |   0.8868
log_avg_wap_realized_vol            |   0.8816 |   0.8816
log_wap_balance_realized_vol        |   0.8663 |   0.8663
price_spread_mean                   |   0.7718 |   0.7718
price_spread_avg_mean               |   0.7693 |   0.7693
price_spread_1_mean                 |   0.7576 |   0.7576
wap_balance_mean          

In [27]:
unique_stock_ids = train_data['stock_id'].unique()
print(f"Train data loaded: {train_df.shape}")
print(f"   - Unique stocks: {train_df['stock_id'].nunique()}")
print(f"   - Unique time_ids: {train_df['time_id'].nunique()}")
print(f"   - Target range: [{train_df['target'].min():.6f}, {train_df['target'].max():.6f}]")

# sequential split by time_id
print("\n Creating sequential train/validation split...")
unique_time_ids = sorted(train_df['time_id'].unique())
split_point = int(0.8 * len(unique_time_ids))

train_time_ids = unique_time_ids[:split_point]
valid_time_ids = unique_time_ids[split_point:]

train_data = train_df[train_df['time_id'].isin(train_time_ids)].copy()
valid_data = train_df[train_df['time_id'].isin(valid_time_ids)].copy()



Train data loaded: (428932, 3)
   - Unique stocks: 112
   - Unique time_ids: 3830
   - Target range: [0.000105, 0.070321]

 Creating sequential train/validation split...


In [ ]:
all_features_list = [] # Initialize list to store features for all stock_id, time_id pairs

for stock_id in unique_stock_ids:
    print(f"Processing stock_id: {stock_id}")

    # Load book and trade data for this stock ID
    stock_book_path = f'{book_train_path}/stock_id={stock_id}'
    stock_trade_path = f'{trade_train_path}/stock_id={stock_id}'

    try:
        book_df = pd.read_parquet(stock_book_path)
        trade_df = pd.read_parquet(stock_trade_path)

        # Filter train_data for the current stock_id and time_ids in the training set
        stock_train_data = train_data[train_data['stock_id'] == stock_id].copy()

        # Loop through time_ids for the current stock
        for _, row in stock_train_data.iterrows():
            time_id = int(row['time_id'])
            target = row['target']

            # Filter book and trade data for the current time_id
            book_slice = book_df[book_df['time_id'] == time_id].copy()
            trade_slice = trade_df[trade_df['time_id'] == time_id].copy()

            # Engineer features
            features = engineer_features_for_stock_time(stock_id, time_id)

            if features:
                # Add the target to the features dictionary
                features['target'] = target
                all_features_list.append(features)

    except Exception as e:
        print(f"❌ Error processing stock_id={stock_id}: {e}")
        continue # Continue to the next stock_id if an error occurs

# Convert the list of feature dictionaries to a DataFrame
training_table = pd.DataFrame(all_features_list)

# Display the head and shape of the resulting training table
print("\n Training Table Head:")
display(training_table.head())
print("\n Training Table Shape:")
print(training_table.shape)

parquet_output_file_path = 'training_table.parquet'
csv_output_file_path = 'training_table.csv'

try:
    # Save as Parquet
    training_table.to_parquet(parquet_output_file_path, index=False)
    print(f"\nTraining table successfully saved to '{parquet_output_file_path}'")

    # Save as CSV
    training_table.to_csv(csv_output_file_path, index=False)
    print(f"Training table successfully saved to '{csv_output_file_path}'")

except Exception as e:
    print(f"Error saving training table: {e}")

Processing stock_id: 0
Processing stock_id: 1
Processing stock_id: 2
Processing stock_id: 3
Processing stock_id: 4
Processing stock_id: 5
Processing stock_id: 6
Processing stock_id: 7
Processing stock_id: 8
Processing stock_id: 9
Processing stock_id: 10
Processing stock_id: 11
Processing stock_id: 13
Processing stock_id: 14
Processing stock_id: 15
Processing stock_id: 16
Processing stock_id: 17
Processing stock_id: 18
Processing stock_id: 19
Processing stock_id: 20
Processing stock_id: 21
Processing stock_id: 22
Processing stock_id: 23
Processing stock_id: 26
Processing stock_id: 27
Processing stock_id: 28


In [ ]:
display(training_table.tail())